<a href="https://colab.research.google.com/github/hucarlos08/Geo-GEE/blob/main/BiTemporalChanges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bitemporal Change Detection using Normalized Burn Ratio (NBR)

This notebook explores a fundamental remote sensing technique: **Change Detection**. The goal is to identify and map differences in landscape conditions by comparing satellite images acquired at different points in time.

## Introduction to Change Detection

Change detection is crucial for monitoring environmental dynamics, managing natural resources, and assessing the impact of events. Examples include:

*   Tracking deforestation or reforestation.
*   Mapping the extent and severity of wildfires or volcanic eruptions.
*   Monitoring urban expansion.
*   Observing changes in water bodies or coastlines.
*   Assessing agricultural practices like harvesting.

The basic premise underlying many change detection methods is that significant changes on the landscape will result in measurable differences in the spectral reflectance values captured by satellite sensors between two dates.

## Challenges in Change Detection

While the concept is straightforward, accurately detecting *meaningful* change requires distinguishing it from various sources of "noise" or non-target variations in the spectral signal. These can include:

*   **Seasonal variations & Phenology:** Changes in vegetation greenness due to normal seasonal cycles.
*   **Atmospheric Effects:** Differences in haze, aerosols, or water vapor between dates.
*   **Illumination Differences:** Variations in sun angle and topography.
*   **Sensor Differences:** Variations between different satellite sensors or even calibration changes over time.
*   **Image Misregistration:** Imperfect alignment of pixels between images.
*   **Clouds and Shadows:** Obscuring the land surface and changing illumination.

Simple two-date differencing methods work best for detecting abrupt, relatively long-lived changes over large areas, where the change signal is strong compared to the noise.

## Methodology: Two-Date NBR Differencing

This exercise focuses on a common and effective method for detecting changes, particularly those related to vegetation health, stress, or removal (like fire impacts): **differencing the Normalized Burn Ratio (NBR)** between a pre-event and a post-event image.

1.  **Normalized Burn Ratio (NBR):**
NBR is a spectral index calculated from the Near-Infrared (NIR) and Shortwave Infrared (SWIR) bands. For Landsat 8, we typically use SWIR2 (Band 7).
$$
NBR = \frac{(NIR - SWIR2)}{(NIR + SWIR2)}
$$
Rationale: Healthy vegetation reflects strongly in the NIR and absorbs in the SWIR, leading to high NBR values. Burned areas, bare soil, or stressed/dry vegetation have lower NIR and higher SWIR reflectance, resulting in lower NBR values. Values range from -1 to +1.

2.  **Difference NBR (dNBR or ΔNBR):**
    *   The core of the change analysis is calculating the difference between NBR maps from two dates.
    *   Formula:
$$ dNBR = NBR_{post} - NBR_{pre} $$
    *   Interpretation:
        *   **Negative dNBR values:** Indicate a decrease in NBR, strongly suggesting vegetation loss or stress (e.g., fire damage, logging). Larger negative values typically correspond to more severe damage.
        *   **Values near zero:** Indicate little or no significant change in NBR between the two dates (stable areas).
        *   **Positive dNBR values:** Indicate an increase in NBR, suggesting vegetation regrowth or increased greenness.

3.  **Thresholding:**
    *   To create a thematic map, the continuous dNBR values are often classified into discrete categories (e.g., High Severity Loss, Low Severity Loss, Stable, Regrowth) by applying specific **thresholds**. Selecting appropriate thresholds is crucial and often requires calibration or knowledge of the specific event and region.

## Workflow Overview

This notebook will guide you through the following steps:

1.  **Image Preparation:** Load Landsat 8 Surface Reflectance data, define pre- and post-event time periods, filter images based on location and cloud cover, select the best image for each period, and apply necessary scaling/renaming.
2.  **Visualization:** Create and examine false-color composites of the pre- and post-event images to visually identify potential changes.
3.  **NBR Calculation:** Compute the NBR index for both the pre- and post-event images.
4.  **dNBR Calculation:** Calculate the difference image (dNBR) by subtracting the pre-event NBR from the post-event NBR.
5.  **Change Classification:** Apply thresholds to the dNBR image to create a classified map showing areas of loss, stability, and gain.
6.  **Map Display:** Visualize the dNBR and classified change maps using appropriate color palettes.

## Dataset

We will use **Landsat 8 Collection 2, Level 2 Surface Reflectance** data (`LANDSAT/LC08/C02/T1_L2`). Using Surface Reflectance (SR) is important as it corrects for atmospheric effects, providing more comparable reflectance values between different dates.

## Learning Objectives

*   Create and interpret false-color composites for visual assessment.
*   Calculate the Normalized Burn Ratio (NBR) index.
*   Create a difference image (dNBR) between two time periods.
*   Produce a classified change map using thresholding on the dNBR image.

## Key GEE Concepts

*   `ee.ImageCollection`, `.filterDate()`, `.filterBounds()`, `.sort()`, `.first()`
*   `.select()` with band renaming
*   Applying scaling factors to SR data
*   `.normalizedDifference()` for index calculation
*   `.subtract()` for image differencing
*   `ee.Image.where()` for conditional reclassification/thresholding
*   Visualization parameters (`visParams`) including palettes
*   Map display using Folium and the `add_ee_layer` helper function

## Setup

In [ ]:
"""
Notebook for Bitemporal Change Detection using NBR with Landsat 8
"""

# Import necessary libraries
import ee
import folium

# Helper function to add GEE tile layers to a Folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
  """Adds a Google Earth Engine tile layer to a Folium map."""
  if ee_image_object is None:
      print(f"Warning: Skipping layer '{name}' because the Earth Engine Image object is null.")
      return self
  try:
      map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
      folium.raster_layers.TileLayer(
          tiles=map_id_dict['tile_fetcher'].url_format,
          attr='Map Data © <a href="https://earthengine.google.com/">Google Earth Engine</a>',
          name=name,
          overlay=True,
          control=True
      ).add_to(self)
      print(f"Layer '{name}' added successfully.")
  except Exception as e:
      print(f"ERROR adding layer '{name}': {e}")
  return self

# Apply the helper function to folium.Map
folium.Map.add_ee_layer = add_ee_layer

# Authenticate and Initialize Google Earth Engine
try:
    ee.Initialize(project='ee-cimat')
    print("Google Earth Engine initialized successfully.")
except Exception as e:
    print(f"Earth Engine initialization failed: {e}")
    print("Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project='ee-cimat')
    print("Google Earth Engine authenticated and initialized successfully.")

 ## Configuration & Initial Data Loading

In [ ]:
# --- Configuration ---

# Dataset: Landsat 8 Collection 2, Level 2 (Surface Reflectance)
landsat_collection_id = 'LANDSAT/LC08/C02/T1_L2'

# Point of Interest (Example from GEE: Southern Oregon)
# Coordinates: [longitude, latitude]
poi_coords = [-123.64, 42.96]

# Date Ranges for Pre- and Post-Event Images
# Example: Pre-fire (June 2013), Post-fire (June 2020) - Adjust as needed
pre_start_date = '2013-06-01'
pre_end_date = '2013-06-30'
post_start_date = '2020-06-01'
post_end_date = '2020-06-30'

# Define band names for Landsat 8 SR (Bands 2-7)
# Original names in the collection
original_bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
# New names for easier reference
new_band_names = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']

print("Configuration set.")

# --- Data Loading and Preprocessing ---

# Load the Image Collection
landsat8_sr = landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').select(original_bands, new_band_names)

print(f"Loaded collection: {landsat_collection_id}")



# Create the Point of Interest Geometry
point = ee.Geometry.Point(poi_coords)
print(f"Point of Interest created at: {poi_coords}")

## Select Pre- and Post-Event Images

In [ ]:
# Cell 3: Select Pre- and Post-Event Images (REVISED CHECK v3)

# --- Select Pre-Event Image ---
print("Requesting pre-event image selection from GEE...")
preImage = landsat8_sr \
    .filterBounds(point) \
    .filterDate(pre_start_date, pre_end_date) \
    .sort('CLOUD_COVER') \
    .first() # Get the least cloudy image in the date range

# --- Select Post-Event Image ---
print("Requesting post-event image selection from GEE...")
postImage = landsat8_sr \
    .filterBounds(point) \
    .filterDate(post_start_date, post_end_date) \
    .sort('CLOUD_COVER') \
    .first() # Get the least cloudy image in the date range

images_found = True

## Create Map & Visualize False-Color

In [ ]:
# --- Visualization Parameters for False-Color (SWIR2, NIR, Red) ---
# Using scaled Surface Reflectance (approx 0-1 range)
false_color_vis = {
'bands': ['swir2', 'nir', 'red'],
'min': 7750,
'max': 22200
};

print("Defined false-color visualization parameters.")

# --- Create Folium Map Object ---
# Center the map on the Point of Interest
map_center_coords = point.coordinates().getInfo()[::-1] # Reverse coords for Folium [lat, lon]
zoom_level = 11 # Adjust zoom level as needed

map_change = folium.Map(location=map_center_coords, zoom_start=zoom_level)
print(f"Folium map initialized, centered at {map_center_coords}.")

# --- Add False-Color Layers to Map ---
map_change = add_ee_layer(map_change, preImage, false_color_vis, 'False Color (Pre-Event)')
map_change = add_ee_layer(map_change, postImage, false_color_vis, 'False Color (Post-Event)')

## Calculate NBR

In [ ]:
# --- Calculate Normalized Burn Ratio (NBR) ---
# NBR = (NIR - SWIR2) / (NIR + SWIR2)

if images_found:
    # Calculate NBR for pre-event image
    nbrPre = preImage.normalizedDifference(['nir', 'swir2']).rename('nbr_pre')

    # Calculate NBR for post-event image
    nbrPost = postImage.normalizedDifference(['nir', 'swir2']).rename('nbr_post')

    print("Calculated NBR for pre- and post-event images.")
else:
    print("Skipping NBR calculation because images were not found.")
    nbrPre = None
    nbrPost = None

## Calculate dNBR & Visualize

In [ ]:
# dNBR = NBR_post - NBR_pre
if nbrPre and nbrPost: # Check if NBR images were calculated
    diff = nbrPost.subtract(nbrPre).rename('dNBR')
    print("Calculated dNBR image.")

    # --- Define dNBR Visualization Parameters ---
    # Using palette from the text (adjust min/max if needed)
    palette_dNBR = [
        '011959', '0E365E', '1D5561', '3E6C55', '687B3E', # Blue to Green (Negative change/loss)
        'B4AD5B', # Near zero transition (example, might need adjustment)
        'D59448', 'F9A380', 'FDB7BD', 'FACCFA' # Orange to Pink (Positive change/gain)
    ]
    # Adjust min/max based on observed dNBR range, +/- 0.2 to +/- 0.5 is common
    dNBR_vis = {
      'palette': palette_dNBR,
      'min': -0.3,
      'max': 0.3
    }
    print("Defined dNBR visualization parameters.")

    # --- Add dNBR Layer to Map ---
    map_change = add_ee_layer(map_change, diff, dNBR_vis, 'dNBR (Post - Pre)')
else:
    print("Skipping dNBR calculation and visualization.")
    diff = None

## Classify Change & Visualize



In [ ]:
# --- Define Thresholds for Classification ---
# These values are examples and might need adjustment based on the specific
# event severity, ecosystem, and desired sensitivity.
# Negative threshold: Pixels below this are considered 'Loss'
# Positive threshold: Pixels above this are considered 'Gain'
thresholdLoss = -0.10 # dNBR below this is loss
thresholdGain = 0.10 # dNBR above this is gain
print(f"Classification thresholds set: Loss <= {thresholdLoss}, Gain >= {thresholdGain}")

# --- Classify the dNBR Image ---
if diff: # Check if dNBR image exists
    # Create a base image with value 0 (representing 'No Change' or 'Stable')
    diffClassified = ee.Image(0).rename('change_class')

    # Classify areas of gain (dNBR >= thresholdGain) as 1
    diffClassified = diffClassified.where(diff.gte(thresholdGain), 1)

    # Classify areas of loss (dNBR <= thresholdLoss) as 2
    # Note: Apply loss *after* gain if thresholds overlap near zero,
    # or ensure they don't overlap if intermediate classes are needed.
    # Here, loss condition overrides gain if a pixel somehow met both (unlikely).
    diffClassified = diffClassified.where(diff.lte(thresholdLoss), 2)

    print("Classified dNBR image into 3 classes (0=Stable, 1=Gain, 2=Loss).")

    # --- Define Classified Change Visualization Parameters ---
    # Palette: 0=Stable (Yellowish), 1=Gain (Blue), 2=Loss (Red)
    palette_classified = 'fcffc8,2659eb,fa1373' # Hex codes: Stable, Gain, Loss
    changeVis = {
      'palette': palette_classified,
      'min': 0,
      'max': 2
    }
    print("Defined classified change visualization parameters.")

    # --- Add Classified Layer to Map (Masked) ---
    # Use selfMask() to make pixels with value 0 (Stable) transparent
    map_change = add_ee_layer(map_change, diffClassified.selfMask(), changeVis, 'Classified Change (Masked)')

    # --- Optional: Add Unmasked Classified Layer ---
    # map_change = add_ee_layer(map_change, diffClassified, changeVis, 'Classified Change (Unmasked)')

else:
    print("Skipping classification because dNBR image is missing.")

##  Display Final Map

In [ ]:
# --- Add Layer Control and Display Map ---

# Add layer control to toggle layers on/off
map_change.add_child(folium.LayerControl())
print("Added Layer Control.")

# Display the final map
print("Displaying map...")
display(map_change)
print("--- Analysis Complete ---")